In [5]:
import pandas as pd
from dateutil.relativedelta import relativedelta

#### `count_months_between_dates`

In [7]:
def count_months_between_dates(date1, date2):
    delta = relativedelta(date2, date1)
    total_months = delta.years * 12 + delta.months
    if delta.days > 0:
        total_months += 1
    return total_months

#### `distribute_values_fiscal_year`

In [98]:
def distribute_values_fiscal_year(row):
    start = row['period_of_performance_start_date']
    
    if pd.isna(start) or start == 'NA':
        year = int(str(row['action_date_fiscal_year'])[:-2])
        start = pd.to_datetime('10/1/{}'.format(year))
        row['period_of_performance_start_date'] = pd.to_datetime(start).strftime('%m/%d/%y')

    start = pd.to_datetime(start)
    end = pd.to_datetime(row['period_of_performance_current_end_date'])

    # Calculate the values to distribute
    try:
        obligation = float(row['federal_action_obligation'].replace('$', '').replace(',', ''))
        total_obligated = float(row['total_obligated_amount'].replace('$', '').replace(',', ''))
        total_face_value = row['total_face_value_of_loan']
    except ValueError:
        return None  # Skip rows with invalid numeric values

    # Calculate the number of months between start and end dates
    months = int((end - start).days / 30)  # Assuming a month has 30 days

    obligation_per_month = obligation / months
    total_obligated_per_month = total_obligated / months
    total_face_value_per_month = total_face_value / months

    # Generate the date range
    date_range = pd.date_range(start=start, periods=months, freq='MS')

    new_rows = []
    for date in date_range:
        new_row = row.copy()
        new_row['period_of_performance_current_end_date'] = date.strftime('%m/%d/%y')
        new_row['federal_action_obligation'] = obligation_per_month
        new_row['total_obligated_amount'] = total_obligated_per_month
        new_row['total_face_value_of_loan'] = total_face_value_per_month
        new_rows.append(new_row)

    return new_rows

In [99]:
usa_spending = pd.read_csv('USA_Spending.csv')

In [102]:
test = usa_spending[:2]
test

,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,federal_action_obligation,total_obligated_amount,total_face_value_of_loan,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,...,recipient_city_name,recipient_zip_code,primary_place_of_performance_scope,primary_place_of_performance_city_name,primary_place_of_performance_zip_4,cfda_number,cfda_title,assistance_type_code,assistance_type_description,prime_award_base_transaction_description
0,7522_H79SM087638_H79SM087638-4182940722_93.243...,ASST_NON_H79SM087638_7522,H79SM087638,"$800,000.00","$800,000.00",0.0,6/28/23,2023.0,NaN,9/29/28,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),SAN DIEGO PROJECT LAUNCH - THE YMCA OF SAN DIE...
1,7522_H79SM084882_H79SM084882-612937641_93.243_003,ASST_NON_H79SM084882_7522,H79SM084882,$0.00,"$1,200,000.00",0.0,6/26/23,2023.0,NaN,9/29/26,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),YMCA'S FINDING YOUTH RESILIENCE PROJECT - THE ...


In [101]:
expanded_data = test.apply(distribute_values_fiscal_year, axis=1)
expanded_data = pd.DataFrame([item for sublist in expanded_data for item in sublist])
expanded_data

,assistance_transaction_unique_key,assistance_award_unique_key,award_id_fain,federal_action_obligation,total_obligated_amount,total_face_value_of_loan,action_date,action_date_fiscal_year,period_of_performance_start_date,period_of_performance_current_end_date,...,recipient_city_name,recipient_zip_code,primary_place_of_performance_scope,primary_place_of_performance_city_name,primary_place_of_performance_zip_4,cfda_number,cfda_title,assistance_type_code,assistance_type_description,prime_award_base_transaction_description
0,7522_H79SM087638_H79SM087638-4182940722_93.243...,ASST_NON_H79SM087638_7522,H79SM087638,13333.333333,13333.333333,0.0,6/28/23,2023.0,10/01/23,10/01/23,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),SAN DIEGO PROJECT LAUNCH - THE YMCA OF SAN DIE...
0,7522_H79SM087638_H79SM087638-4182940722_93.243...,ASST_NON_H79SM087638_7522,H79SM087638,13333.333333,13333.333333,0.0,6/28/23,2023.0,10/01/23,11/01/23,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),SAN DIEGO PROJECT LAUNCH - THE YMCA OF SAN DIE...
0,7522_H79SM087638_H79SM087638-4182940722_93.243...,ASST_NON_H79SM087638_7522,H79SM087638,13333.333333,13333.333333,0.0,6/28/23,2023.0,10/01/23,12/01/23,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),SAN DIEGO PROJECT LAUNCH - THE YMCA OF SAN DIE...
0,7522_H79SM087638_H79SM087638-4182940722_93.243...,ASST_NON_H79SM087638_7522,H79SM087638,13333.333333,13333.333333,0.0,6/28/23,2023.0,10/01/23,01/01/24,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),SAN DIEGO PROJECT LAUNCH - THE YMCA OF SAN DIE...
0,7522_H79SM087638_H79SM087638-4182940722_93.243...,ASST_NON_H79SM087638_7522,H79SM087638,13333.333333,13333.333333,0.0,6/28/23,2023.0,10/01/23,02/01/24,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),SAN DIEGO PROJECT LAUNCH - THE YMCA OF SAN DIE...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,7522_H79SM084882_H79SM084882-612937641_93.243_003,ASST_NON_H79SM084882_7522,H79SM084882,0.000000,33333.333333,0.0,6/26/23,2023.0,10/01/23,05/01/26,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),YMCA'S FINDING YOUTH RESILIENCE PROJECT - THE ...
1,7522_H79SM084882_H79SM084882-612937641_93.243_003,ASST_NON_H79SM084882_7522,H79SM084882,0.000000,33333.333333,0.0,6/26/23,2023.0,10/01/23,06/01/26,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),YMCA'S FINDING YOUTH RESILIENCE PROJECT - THE ...
1,7522_H79SM084882_H79SM084882-612937641_93.243_003,ASST_NON_H79SM084882_7522,H79SM084882,0.000000,33333.333333,0.0,6/26/23,2023.0,10/01/23,07/01/26,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),YMCA'S FINDING YOUTH RESILIENCE PROJECT - THE ...
1,7522_H79SM084882_H79SM084882-612937641_93.243_003,ASST_NON_H79SM084882_7522,H79SM084882,0.000000,33333.333333,0.0,6/26/23,2023.0,10/01/23,08/01/26,...,SAN DIEGO,92123.0,SINGLE ZIP CODE,SAN DIEGO,92123,93.243,SUBSTANCE ABUSE AND MENTAL HEALTH SERVICES PRO...,4.0,PROJECT GRANT (B),YMCA'S FINDING YOUTH RESILIENCE PROJECT - THE ...
